In [1]:
import pandas as pd
import numpy as np
import psycopg2
import os
from dotenv import load_dotenv
import sys #permite navegar por el sistema
sys.path.append("../") #solo aplica al soporte
import os
from dotenv import load_dotenv
from src.etl import load as lo #con jupyter
import unicodedata
from src.etl import transform as tr #con jupyter

In [2]:
load_dotenv()

True

In [3]:
#Llamamos a la variable de entorno
DB_NAME = os.getenv("DB_NAME")
DB_USER=os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_HOST= os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")

In [4]:
conn, cur = lo.crear_conexión()

In [5]:
#Comprobamos que la conexión está bien realizada:
cur.execute("SELECT version();")
cur.fetchone() #para que muestre el resultado de la query anterior

('PostgreSQL 16.4, compiled by Visual C++ build 1940, 64-bit',)

In [6]:
lo.cerrar_conexion(conn, cur)

# Creación de los datos para las tablas:

In [7]:
#Importo los df procesados:
ARCHIVO_GUARDAR_ESCRAPEO_VIAJES_PROCESADOS=os.getenv('ARCHIVO_GUARDAR_ESCRAPEO_VIAJES_PROCESADOS')
ARCHIVO_GUARDAR_CONTINENTES_PROCESADOS =os.getenv('ARCHIVO_GUARDAR_CONTINENTES_PROCESADOS')
ARCHIVO_GUARDAR_ITINERARIOS_PROCESADOS_1=os.getenv('ARCHIVO_GUARDAR_ITINERARIOS_PROCESADOS_1')
ARCHIVO_GUARDAR_DATOS_API_PROCESADOS=os.getenv('ARCHIVO_GUARDAR_DATOS_API_PROCESADOS')
ARCHIVO_GUARDAR_ESCRAPEO_API_OPENCAGE=os.getenv('ARCHIVO_GUARDAR_ESCRAPEO_API_OPENCAGE')
ARCHIVO_GUARDAR_TOTAL_CIUDADES_API = os.getenv('ARCHIVO_GUARDAR_TOTAL_CIUDADES_API')

## 1. Tabla pais_destino

###  1.1 Preparación de los datos: Unión de los paises y continentes tanto escrapeados como obtenidos de la API de turismo emisor y de las ciudades de los itinerarios

#### a) Turismo emisor

In [8]:
df_turismo_emisor_procesado=pd.read_csv(ARCHIVO_GUARDAR_DATOS_API_PROCESADOS)
df_turismo_emisor_procesado.columns

Index(['Unnamed: 0', 'AÑO', 'MES', 'CCAA_ORIGEN', 'CONTINENTE_DESTINO',
       'PAIS_DESTINO', 'TURISTAS', 'PERNOCTACIONES', 'ESTANCIA_MEDIA'],
      dtype='object')

In [9]:
df_turismo_emisor_procesado.head()

,Unnamed: 0,AÑO,MES,CCAA_ORIGEN,CONTINENTE_DESTINO,PAIS_DESTINO,TURISTAS,PERNOCTACIONES,ESTANCIA_MEDIA
0,0,2019,7,Andalucía,América,Argentina,"419,0","3616,0","8,6"
1,1,2019,7,Andalucía,América,Bolivia,"61,0","486,0","8,0"
2,2,2019,7,Andalucía,América,Brasil,"277,0","2214,0","8,0"
3,3,2019,7,Andalucía,América,Canadá,"483,0","6159,0","12,8"
4,4,2019,7,Andalucía,América,Chile,"242,0","2068,0","8,5"


In [10]:
df_paises_continentes_turismos_emisor=df_turismo_emisor_procesado[['PAIS_DESTINO','CONTINENTE_DESTINO']]
df_paises_continentes_turismos_emisor=df_paises_continentes_turismos_emisor.drop_duplicates()
df_paises_continentes_turismos_emisor.CONTINENTE_DESTINO.unique()

array(['América', 'Asia', 'Europa', 'Oceanía', 'África'], dtype=object)

In [11]:
df_paises_continentes_turismos_emisor.columns

Index(['PAIS_DESTINO', 'CONTINENTE_DESTINO'], dtype='object')

In [12]:
df_paises_continentes_turismos_emisor= df_paises_continentes_turismos_emisor.rename(
    columns={'PAIS_DESTINO':'nombre_pais_destino', 'CONTINENTE_DESTINO':'nombre_continente'})
df_paises_continentes_turismos_emisor= df_paises_continentes_turismos_emisor.sort_values(by='nombre_pais_destino').reset_index(drop=True)

In [13]:
df_paises_continentes_turismos_emisor

,nombre_pais_destino,nombre_continente
0,Albania,Europa
1,Alemania,Europa
2,Andorra,Europa
3,Angola,África
4,Antigua,América
...,...,...
153,Venezuela,América
154,Vietnam,Asia
155,Yemen,Asia
156,Zambia,África


In [14]:
#normalizo y capitalizo la columna nombre_pais_destino:
df_paises_continentes_turismos_emisor['nombre_pais_destino'] = df_paises_continentes_turismos_emisor['nombre_pais_destino'].map(tr.normalizar_texto)
df_paises_continentes_turismos_emisor['nombre_pais_destino'] = df_paises_continentes_turismos_emisor['nombre_pais_destino'].map(tr.capitalizar_texto)

#### b) Paises escrapeados

In [15]:
df_paises_continentes_escrapeo = pd.read_pickle(ARCHIVO_GUARDAR_CONTINENTES_PROCESADOS)
df_paises_continentes_escrapeo.columns

Index(['continente', 'pais', 'fecha_escrapeo', 'en_ultimo_escrapeo'], dtype='object')

In [16]:
df_paises_continentes_escrapeo

,continente,pais,fecha_escrapeo,en_ultimo_escrapeo
0,Europa,Albania,2025-04-14,Si
1,Europa,Alemania,2025-04-14,Si
2,América,Antigua,2025-04-14,Si
3,América,Argentina,2025-04-14,Si
4,América,Aruba,2025-04-14,Si
...,...,...,...,...
3792,América,Uruguay,2025-05-18,Si
3793,Asia,Uzbekistán,2025-05-18,Si
3794,Asia,Vietnam,2025-05-18,Si
3795,África,Zanzíbar,2025-05-18,Si


In [17]:
df_paises_continentes_escrapeo=df_paises_continentes_escrapeo[['pais','continente']]
df_paises_continentes_escrapeo= df_paises_continentes_escrapeo.drop_duplicates()

In [18]:
df_paises_continentes_escrapeo= df_paises_continentes_escrapeo.rename(
    columns={'pais':'nombre_pais_destino', 'continente':'nombre_continente'})

In [19]:
df_paises_continentes_escrapeo= df_paises_continentes_escrapeo.sort_values(by='nombre_pais_destino').reset_index(drop=True)
df_paises_continentes_escrapeo

,nombre_pais_destino,nombre_continente
0,Albania,Europa
1,Alemania,Europa
2,Antigua,América
3,Arabia Saudí,Asia
4,Argentina,América
...,...,...
120,Uruguay,América
121,Uzbekistán,Asia
122,Vietnam,Asia
123,Zanzíbar,África


In [20]:
#normalizo y capitalizo la columna nombre_pais_destino:
df_paises_continentes_escrapeo['nombre_pais_destino'] = df_paises_continentes_escrapeo['nombre_pais_destino'].map(tr.normalizar_texto)
df_paises_continentes_escrapeo['nombre_pais_destino'] = df_paises_continentes_escrapeo['nombre_pais_destino'].map(tr.capitalizar_texto)

#### c) Paises en itinerarios ciudades

In [21]:
df_itinerarios_ciudades = pd.read_pickle(ARCHIVO_GUARDAR_ITINERARIOS_PROCESADOS_1)
df_itinerarios_ciudades.columns

Index(['itinerario_modificado_para_dividir', 'ciudad', 'pais_correcto'], dtype='object')

In [22]:
df_itinerarios_ciudades_pais_continentes = pd.DataFrame()
df_itinerarios_ciudades_pais_continentes['pais'] = df_itinerarios_ciudades['pais_correcto'].drop_duplicates().reset_index(drop=True)

In [23]:
df_itinerarios_ciudades_pais_continentes

,pais
0,Albania
1,Montenegro
2,Alemania
3,Austria
4,Bélgica
...,...
130,Armenia
131,Arabia Saudi
132,Bosnia y Herzegovina
133,Taiwán


In [24]:
#normalizo y capitalizo la columna pais:
df_itinerarios_ciudades_pais_continentes['pais'] = df_itinerarios_ciudades_pais_continentes['pais'].map(tr.normalizar_texto)
df_itinerarios_ciudades_pais_continentes['pais'] = df_itinerarios_ciudades_pais_continentes['pais'].map(tr.capitalizar_texto)

In [25]:
df_opencage_ciudades_totales = pd.read_pickle(ARCHIVO_GUARDAR_TOTAL_CIUDADES_API)
df_opencage_ciudades_totales.columns

Index(['ciudad', 'continente_api', 'pais_api', 'latitud', 'longitud'], dtype='object')

In [26]:
df_opencage_ciudades_totales.continente_api.value_counts()

continente_api
Europe                     755
North America              398
Asia                       377
Africa                     207
South America              150
Información desconocida    126
Oceania                     87
Name: count, dtype: int64

In [27]:
#sustituyo los valores dados en Inglés por los valores en Español en la columna continente_api:
df_opencage_ciudades_totales.continente_api= df_opencage_ciudades_totales.continente_api.replace({'North America':'América',
                                                'South America': 'América',
                                                'Africa': 'África',
                                                'Europe': 'Europa',
                                                'Oceania':'Oceanía'})

In [28]:
#normalizo y capitalizo la columna pais_api:
df_opencage_ciudades_totales['pais_api'] = df_opencage_ciudades_totales['pais_api'].map(tr.normalizar_texto)
df_opencage_ciudades_totales['pais_api'] = df_opencage_ciudades_totales['pais_api'].map(tr.capitalizar_texto)

In [29]:
df_opencage_ciudades_totales

,ciudad,continente_api,pais_api,latitud,longitud
0,Tirana,Europa,Albania,41.328148,19.818444
1,Dürres,Europa,Albania,41.313255,19.446235
2,Berat,Europa,Albania,40.701864,19.958076
3,Kruja,Europa,Albania,41.511251,19.792852
4,Shkodër,Europa,Albania,42.068137,19.512144
...,...,...,...,...,...
1471,Mesa Verde,América,Estados Unidos De America,33.605858,-114.731072
1472,Great Sand,Oceanía,Australia,-38.262747,144.775661
1473,Dunes,Europa,Francia,44.087932,0.769998
1474,Pueblo,América,Estados Unidos De America,38.263995,-104.614187


In [30]:
ARCHIVO_EXTRACCION_API_DUPLICADOS = os.getenv('ARCHIVO_EXTRACCION_API_DUPLICADOS')
df_paises_api_duplicados = pd.read_pickle(ARCHIVO_EXTRACCION_API_DUPLICADOS)
df_paises_api_duplicados.continente_api= df_paises_api_duplicados.continente_api.replace({'North America':'América',
                                                'South America': 'América',
                                                'Africa': 'África',
                                                'Europe': 'Europa',
                                                'Oceania':'Oceanía'})

In [31]:
dict_paises_continentes_ciudades = {'pais':[], 'continente':[]}
for pais in df_itinerarios_ciudades_pais_continentes.pais:
    if pais in df_paises_continentes_escrapeo.nombre_pais_destino.tolist():
        pass
    else:
        if pais in df_paises_continentes_turismos_emisor.nombre_pais_destino.tolist():
           pass
        else:
           if pais in df_opencage_ciudades_totales.pais_api.tolist():
            #print(f'{pais} ya está en continentes opencage')
            dict_paises_continentes_ciudades['pais'].append(pais)
            continente = (df_opencage_ciudades_totales[df_opencage_ciudades_totales.pais_api == pais]).continente_api.unique()[0]
            dict_paises_continentes_ciudades['continente'].append(continente)
           else:
                if pais in df_paises_api_duplicados.pais_api.tolist():
                    #print(f'{pais} ya está en continentes opencage')
                    dict_paises_continentes_ciudades['pais'].append(pais)
                    continente = (df_paises_api_duplicados[df_paises_api_duplicados.pais_api == pais]).continente_api.unique()[0]
                    dict_paises_continentes_ciudades['continente'].append(continente)
                else:
                    print(f'{pais} ERROR')
df_paises_continentes_ciudades = pd.DataFrame(dict_paises_continentes_ciudades)
df_paises_continentes_ciudades

,pais,continente
0,Chequia,Europa
1,Sovereign Base Areas Of Akrotiri And Dhekelia,Asia
2,No Existe Resultado,No Existe Resultado
3,Catar,Asia


In [32]:
df_paises_continentes_ciudades= df_paises_continentes_ciudades.rename(
    columns={'pais':'nombre_pais_destino', 'continente':'nombre_continente'})

In [33]:
df_paises_continentes_ciudades

,nombre_pais_destino,nombre_continente
0,Chequia,Europa
1,Sovereign Base Areas Of Akrotiri And Dhekelia,Asia
2,No Existe Resultado,No Existe Resultado
3,Catar,Asia


#### d) Uno los 3 DF para conseguir una única tabla con todos los paises y continentes

In [34]:
df_paises_continentes_totales=pd.concat(
    [df_paises_continentes_turismos_emisor,df_paises_continentes_escrapeo, df_paises_continentes_ciudades], axis=0)
df_paises_continentes_totales = df_paises_continentes_totales.drop_duplicates(['nombre_pais_destino', 'nombre_continente']).reset_index(drop=True)
df_paises_continentes_totales= df_paises_continentes_totales.sort_values(by='nombre_pais_destino').reset_index(drop=True)
df_paises_continentes_totales



,nombre_pais_destino,nombre_continente
0,Albania,Europa
1,Alemania,Europa
2,Andorra,Europa
3,Angola,África
4,Antigua,América
...,...,...
185,Vietnam,Asia
186,Yemen,Asia
187,Zambia,África
188,Zanzibar,África


In [35]:
#reviso los paises que aparecen asociados a más de un continente:
df_contador_paises_duplicados = pd.DataFrame(df_paises_continentes_totales.nombre_pais_destino.value_counts()).reset_index()
df_contador_paises_duplicados =df_contador_paises_duplicados[df_contador_paises_duplicados['count']>1]
df_contador_paises_duplicados

,nombre_pais_destino,count
0,Armenia,2
1,Turquia,2
2,Georgia,2


In [36]:
lista_paises_duplicados = df_contador_paises_duplicados.nombre_pais_destino.tolist()
lista_paises_duplicados

['Armenia', 'Turquia', 'Georgia']

In [37]:
#Elimino los indices incorrectos:
lista_indices_eliminar = []
for indice,pais in enumerate(df_paises_continentes_totales.nombre_pais_destino):
    if pais in lista_paises_duplicados:
        df_paises_continentes_totales.loc[indice, "nombre_continente"]  = 'Asia'

df_paises_continentes_totales = df_paises_continentes_totales.drop_duplicates()

In [38]:
df_paises_continentes_totales.nombre_pais_destino.value_counts()

nombre_pais_destino
Albania     1
Alemania    1
Andorra     1
Angola      1
Antigua     1
           ..
Vietnam     1
Yemen       1
Zambia      1
Zanzibar    1
Zimbabue    1
Name: count, Length: 187, dtype: int64

## 1.2 Carga en BBDD

La tabla pais_destino tiene que tener los siguientes campos:
- id_pais_destino SERIAL PRIMARY KEY,
- nombre_pais_destino TEXT UNIQUE,
- nombre_continente TEXT

In [39]:
#Extraigo la información de la BBDD
query_extraccion = "SELECT nombre_pais_destino, id_pais_destino FROM pais_destino"
pais_destino_dict=lo.extraer_datos_de_BBDD(query_extraccion)

In [40]:
#Creo los datos para insertarlos en la BBDD:
data_to_insert=[]
for _,row in df_paises_continentes_totales.iterrows():
    nombre_pais_destino = row["nombre_pais_destino"]
    if nombre_pais_destino in  pais_destino_dict.keys():
        pass
    else: 
        nombre_continente = row["nombre_continente"]
        data_to_insert.append([nombre_pais_destino, nombre_continente])

In [41]:
data_to_insert

[]

In [42]:
#Subo la información a la BBDD:
insert_query = """
INSERT INTO pais_destino(nombre_pais_destino, nombre_continente)
VALUES (%s, %s)
"""
lo.insertar_datos_en_BBDD(insert_query, data_to_insert)

print(f'Se han añadido {len(data_to_insert)} paises en la tabla pais_destino')

Se han añadido 0 paises en la tabla pais_destino


In [43]:
#conn.rollback()

## 2. Tabla itinerario

### 2.1 Extraigo la información del df_itinerario_ciudades y me quedo con los itinerarios sin duplicados:

In [44]:
df_itinerarios_ciudades = pd.read_pickle(ARCHIVO_GUARDAR_ITINERARIOS_PROCESADOS_1)
df_itinerarios_ciudades.head()

,itinerario_modificado_para_dividir,ciudad,pais_correcto
0,"Tirana, Dürres, Berat, Kruja, Shkodër, Petrova...",Tirana,Albania
1,"Tirana, Dürres, Berat, Kruja, Shkodër, Petrova...",Durres,Albania
2,"Tirana, Dürres, Berat, Kruja, Shkodër, Petrova...",Berat,Albania
3,"Tirana, Dürres, Berat, Kruja, Shkodër, Petrova...",Kruja,Albania
4,"Tirana, Dürres, Berat, Kruja, Shkodër, Petrova...",Shkoder,Albania


In [45]:
#limpio los espacios entre las comas que sobran
df_itinerarios_ciudades['itinerario_modificado_para_dividir'] = df_itinerarios_ciudades['itinerario_modificado_para_dividir'].apply(lo.limpiar_texto)

In [46]:
lista_itinerarios_unicos = df_itinerarios_ciudades.itinerario_modificado_para_dividir.sort_values().unique()
len(lista_itinerarios_unicos)

1139

### 2.2 Hago la carga en la BBDD:

La tabla itinerario tiene que tener los siguientes campos:
- id_itinerario SERIAL PRIMARY KEY
- detalle_itinerario TEXT UNIQUE

In [47]:
#Extraigo la información de la BBDD
query_extraccion = "SELECT detalle_itinerario, id_itinerario FROM itinerario"
itinerario_dict=lo.extraer_datos_de_BBDD(query_extraccion)

In [48]:
#Creo los datos para insertarlos en la BBDD:
data_to_insert=[]
for itinerario in lista_itinerarios_unicos:
    if itinerario in itinerario_dict.keys():
        pass
    else: 
        data_to_insert.append((itinerario,))  

In [49]:
#Subo la información a la BBDD:
insert_query = """
INSERT INTO itinerario(detalle_itinerario)
VALUES (%s)
"""
lo.insertar_datos_en_BBDD(insert_query, data_to_insert)

print(f'Se han añadido {len(data_to_insert)} itinerarios nuevos en la tabla pais_destino')

Se han añadido 0 itinerarios nuevos en la tabla pais_destino


## 3. Tabla ciudad

### 3.1 Extraigo la información del df_itinerario_ciudades y me quedo con las ciudades sin duplicados:

In [50]:
df_itinerarios_ciudades.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9366 entries, 0 to 9365
Data columns (total 3 columns):
 #   Column                              Non-Null Count  Dtype 
---  ------                              --------------  ----- 
 0   itinerario_modificado_para_dividir  9366 non-null   object
 1   ciudad                              9366 non-null   object
 2   pais_correcto                       9366 non-null   object
dtypes: object(3)
memory usage: 219.6+ KB


In [51]:
lista_ciudades_unicas = df_itinerarios_ciudades.ciudad.sort_values().unique()
len(lista_ciudades_unicas)

2040

In [52]:
#creo el DF con la ciudad y el pais para preparar los datos para la carga y elimino duplicados:
df_ciudad_pais_unicos = df_itinerarios_ciudades[['ciudad','pais_correcto']].drop_duplicates().reset_index(drop=True)
df_ciudad_pais_unicos

,ciudad,pais_correcto
0,Tirana,Albania
1,Durres,Albania
2,Berat,Albania
3,Kruja,Albania
4,Shkoder,Albania
...,...,...
2035,Dilijan,Armenia
2036,Sanahin Tbilisi,Georgia
2037,Baku,Georgia
2038,Gobustan,Estados Unidos


In [53]:
#Comprueba las ciudades duplicadas
ciudades_duplicadas = []
for ciudad in df_ciudad_pais_unicos.ciudad.unique():
    df_filtrado = df_ciudad_pais_unicos[df_ciudad_pais_unicos.ciudad == ciudad]
    if len(df_filtrado.pais_correcto.unique()) >1:
        ciudades_duplicadas.append(ciudad)
        if len(df_filtrado.pais_correcto.unique()) ==2:
            pais1= df_filtrado.pais_correcto.unique()[-2]
            pais2=df_filtrado.pais_correcto.unique()[-1]
            print(f'La ciudad {ciudad} aparece en los paises {pais1}, {pais2}')
        elif len(df_filtrado.pais_correcto.unique()) ==3:
            pais3= df_filtrado.pais_correcto.unique()[-3]
            pais2= df_filtrado.pais_correcto.unique()[-2]
            pais3=df_filtrado.pais_correcto.unique()[-1]
            print(f'La ciudad {ciudad} aparece en los paises {pais1}, {pais2}, {pais3}')
        else: 
            pais4= df_filtrado.pais_correcto.unique()[-4]
            pais3= df_filtrado.pais_correcto.unique()[-3]
            pais2= df_filtrado.pais_correcto.unique()[-2]
            pais3=df_filtrado.pais_correcto.unique()[-1]
            print(f'La ciudad {ciudad} aparece en los paises {pais1}, {pais2}, {pais3} , {pais4}')


In [54]:
ciudades_duplicadas

[]

### 3.2 Hago la carga en la BBDD:

La tabla ciudad tiene que tener los siguientes campos:
- id_ciudad SERIAL PRIMARY KEY,
- nombre_ciudad TEXT UNIQUE,
- id_pais_destino INT REFERENCES pais_destino(id_pais_destino) ON DELETE CASCADE

In [55]:
#Extraigo la información de la BBDD existente en la tabla Ciudad
query_extraccion = "SELECT nombre_ciudad, id_ciudad FROM ciudad"
ciudad_dict=lo.extraer_datos_de_BBDD(query_extraccion)
print(len(ciudad_dict))

2040


In [56]:
#Extraigo el id_pais_destino de la tabla pais_destino de la BBDD:
query_extraccion = "SELECT nombre_pais_destino, id_pais_destino FROM pais_destino"
pais_dict=lo.extraer_datos_de_BBDD(query_extraccion)
print(len(pais_dict))

187


In [57]:
#Creo los datos para insertarlos en la BBDD:
data_to_insert=[]
for _,row in df_ciudad_pais_unicos.iterrows():
    nombre_ciudad = row["ciudad"]
    if nombre_ciudad in ciudad_dict.keys():
        pass
    else:
        pais_correcto = row["pais_correcto"]
        id_pais_destino = pais_dict.get(pais_correcto)
        data_to_insert.append([nombre_ciudad, id_pais_destino])
print(len(data_to_insert))

0


In [58]:
#Subo la información a la BBDD:
insert_query = """
INSERT INTO ciudad (nombre_ciudad,id_pais_destino)
VALUES (%s, %s)
"""
lo.insertar_datos_en_BBDD(insert_query, data_to_insert)

print(f'Se han añadido {len(data_to_insert)} ciudades nuevas en la tabla ciudad')

Se han añadido 0 ciudades nuevas en la tabla ciudad


## 4. Tabla ciudad_itinerario

### 4.1 Extraigo la información del df_itinerario_ciudades y me quedo con las columnas itinerario y ciudad sin duplicados:

In [59]:
df_itinerarios_ciudades.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9366 entries, 0 to 9365
Data columns (total 3 columns):
 #   Column                              Non-Null Count  Dtype 
---  ------                              --------------  ----- 
 0   itinerario_modificado_para_dividir  9366 non-null   object
 1   ciudad                              9366 non-null   object
 2   pais_correcto                       9366 non-null   object
dtypes: object(3)
memory usage: 219.6+ KB


In [60]:
#elimino duplicados
df_itinerarios_ciudades = df_itinerarios_ciudades.drop_duplicates().reset_index(drop=True)
print(len(df_itinerarios_ciudades))

7422


In [61]:
df_itinerario_ciudad_unicos = df_itinerarios_ciudades[['itinerario_modificado_para_dividir','ciudad']].drop_duplicates().reset_index(drop=True)
print(len(df_itinerario_ciudad_unicos))
print(df_itinerario_ciudad_unicos.columns)

7422
Index(['itinerario_modificado_para_dividir', 'ciudad'], dtype='object')


### 4.2 Hago la carga en la BBDD:

La tabla ciudad_itinerario tiene que tener los siguientes campos:
- id_ciudad INT REFERENCES ciudad(id_ciudad) ON DELETE CASCADE,
- id_itinerario INT REFERENCES itinerario(id_itinerario) ON DELETE CASCADE,
- PRIMARY KEY (id_ciudad, id_itinerario)

In [62]:
#Extraigo la información de la BBDD existente en la tabla Ciudad
query_extraccion = "SELECT nombre_ciudad, id_ciudad FROM ciudad"
ciudad_dict=lo.extraer_datos_de_BBDD(query_extraccion)
print(len(ciudad_dict))

2040


In [63]:
#Extraigo la información de la BBDD
query_extraccion = "SELECT detalle_itinerario, id_itinerario FROM itinerario"
itinerario_dict=lo.extraer_datos_de_BBDD(query_extraccion)
print(len(itinerario_dict))

1139


In [67]:
# Obtengo todas las combinaciones existentes (id_ciudad, id_itinerario) de la tabla
query_extraccion_tupla="SELECT id_ciudad, id_itinerario FROM ciudad_itinerario"
existing_pairs=lo.extraer_tupla_datos_bbdd(query_extraccion_tupla)
print(len(existing_pairs))


7422


In [68]:
#Creo los datos para insertarlos en la BBDD:
data_to_insert=[]
for _,row in df_itinerario_ciudad_unicos.iterrows():
    detalle_itinerario = row["itinerario_modificado_para_dividir"]
    nombre_ciudad = row["ciudad"]
    id_ciudad = ciudad_dict.get(nombre_ciudad)
    id_itinerario = itinerario_dict.get(detalle_itinerario)
    pair = (id_ciudad, id_itinerario) #creo la tupla de combinación de id_ciudad e id_itinerario
    if pair not in existing_pairs:
        data_to_insert.append([id_ciudad, id_itinerario])
print(len(data_to_insert))

0


In [69]:
#Subo la información a la BBDD:
insert_query = """
INSERT INTO ciudad_itinerario (id_ciudad,id_itinerario)
VALUES (%s, %s)
"""
lo.insertar_datos_en_BBDD(insert_query,data_to_insert)
print(f'Se han añadido {len(data_to_insert)} itinerarios nuevos en la tabla itinerario')

Se han añadido 0 itinerarios nuevos en la tabla itinerario


In [ ]:
#conn.rollback()

## 5. Tabla viaje

### 5.1 Extraigo la información de los viajes procesados y elimino los duplicados:

In [70]:
#importo el fichero procesado de viajes
df_viajes_total = pd.read_pickle(ARCHIVO_GUARDAR_ESCRAPEO_VIAJES_PROCESADOS)
df_viajes_total.head()

,pais,nombre_viaje,duracion_viaje,itinerario,precio,url_viaje,fecha_escrapeo,en_ultimo_escrapeo,itinerario_modificado_para_dividir
0,Albania,Albania y Montenegro,8 Días / 7 Noches,"Tirana, Dürres, Berat, Kruja, Shkodër, Petrova...",1755,https://es.tui.com/viaje/albania-y-montenegro,2025-04-07,Si,"Tirana, Dürres, Berat, Kruja, Shkodër, Petrova..."
1,Alemania,Berlín,4 Días / 3 Noches,Berlín,708,https://viajeonline.es.tui.com/es/idea/1463025...,2025-04-07,Si,Berlín
2,Alemania,"Fly & Drive Austria, Entre Lagos y Montañas",8 Días / 7 Noches,"Salzburgo, Gmunden, Linz, Melk, Viena, Graz, H...",827,https://viajeonline.es.tui.com/es/idea/7897512...,2025-04-07,Si,"Salzburgo, Gmunden, Linz, Melk, Viena, Graz, H..."
3,Alemania,Fly & Drive Carlomagno a su aire,10 Días / 9 Noches,"Bruselas, Durbuy, Lieja, Aquisgrán, Colonia, B...",857,https://viajeonline.es.tui.com/es/idea/8404511...,2025-04-07,Si,"Bruselas, Durbuy, Lieja, Aquisgrán, Colonia, B..."
4,Alemania,Fly & Drive Tirol al Completo,8 Días / 7 Noches,"sbruck, Cataratas Krimml, Vipiteno, Bresanona,...",894,https://viajeonline.es.tui.com/es/idea/7906930...,2025-04-07,Si,"sbruck, Cataratas Krimml, Vipiteno, Bresanona,..."


In [71]:
#limpio los datos de la columna itinerario_modificado_para_dividir
df_viajes_total['itinerario_modificado_para_dividir'] = df_viajes_total['itinerario_modificado_para_dividir'].apply(lo.limpiar_texto)

In [72]:
#me quedo con los viajes escrapeados la última vez para indicar si el viaje está aún activo o no:
viajes_ultimo_escrapeo = df_viajes_total[df_viajes_total.fecha_escrapeo == max(df_viajes_total.fecha_escrapeo) ]
lista_nombre_ultimo_escrapeo = viajes_ultimo_escrapeo.nombre_viaje.unique().tolist()
lista_url_ultimo_escrapeo = viajes_ultimo_escrapeo.url_viaje.unique().tolist()
viajes_ultimo_escrapeo

,pais,nombre_viaje,duracion_viaje,itinerario,precio,url_viaje,fecha_escrapeo,en_ultimo_escrapeo,itinerario_modificado_para_dividir
30331,Albania,Albania y Montenegro,8 Días / 7 Noches,"Tirana, Dürres, Berat, Kruja, Shkodër, Petrova...",1755,https://es.tui.com/viaje/albania-y-montenegro,2025-05-18,Si,"Tirana, Dürres, Berat, Kruja, Shkodër, Petrova..."
30332,Albania,Ronda por los Balcanes,15 Días / 14 Noches,"Zagreb, Ljubljana, Postojna, Parque Nacional d...",3485,https://viajeonline.es.tui.com/es/idea/2601015...,2025-05-18,Si,"Zagreb, Ljubljana, Postojna, Parque Nacional d..."
30333,Alemania,Berlín,4 Días / 3 Noches,Berlín,708,https://viajeonline.es.tui.com/es/idea/1463025...,2025-05-18,Si,Berlín
30334,Alemania,"Fly & Drive Austria, Entre Lagos y Montañas",8 Días / 7 Noches,"Salzburgo, Gmunden, Linz, Melk, Viena, Graz, H...",827,https://viajeonline.es.tui.com/es/idea/7897512...,2025-05-18,Si,"Salzburgo, Gmunden, Linz, Melk, Viena, Graz, H..."
30335,Alemania,Fly & Drive Carlomagno a su aire,10 Días / 9 Noches,"Bruselas, Durbuy, Lieja, Aquisgrán, Colonia, B...",857,https://viajeonline.es.tui.com/es/idea/8404511...,2025-05-18,Si,"Bruselas, Durbuy, Lieja, Aquisgrán, Colonia, B..."
...,...,...,...,...,...,...,...,...,...
55768,Zimbabue,Cataratas y Parques de Botsuana y Mauricio,13 Días / 10 Noches,"Cataratas Victoria, PN Chobe, Delta de Okavang...",5369,https://es.tui.com/viaje/cataratas-y-parques-d...,2025-05-18,Si,"Cataratas Victoria, PN Chobe, Delta de Okavang..."
55769,Zimbabue,Cataratas y Parques de Botsuana y Mozambique,13 Días / 10 Noches,"Cataratas Victoria, PN Chobe, Delta de Okavang...",5868,https://es.tui.com/viaje/cataratas-y-parques-d...,2025-05-18,Si,"Cataratas Victoria, PN Chobe, Delta de Okavang..."
55770,Zimbabue,Cataratas y Parques de Botsuana y Maldivas,13 Días / 9 Noches,"Cataratas Victoria, PN Chobe, Delta de Okavang...",6189,https://es.tui.com/viaje/cataratas-y-parques-d...,2025-05-18,Si,"Cataratas Victoria, PN Chobe, Delta de Okavang..."
55771,Zimbabue,Zimbabue Salvaje,11 Días / 8 Noches,"Cataratas Victoria, PN Hwange y Mana Pools",6539,https://es.tui.com/viaje/zimbabue-salvaje,2025-05-18,Si,"Cataratas Victoria, PN Hwange, Mana Pools"


In [73]:
#me quedo con las columnas que me interesan:
df_viajes_unicos = df_viajes_total[[ 'url_viaje','nombre_viaje','duracion_viaje','itinerario_modificado_para_dividir','en_ultimo_escrapeo']]
#elimino duplicados
df_viajes_unicos = df_viajes_unicos.drop_duplicates('url_viaje').reset_index(drop=True)
len(df_viajes_unicos)

1435

In [76]:
for _,row in df_viajes_unicos.iterrows():
    nombre_viaje = row["nombre_viaje"]
    url_viaje = row["url_viaje"]
    if nombre_viaje in lista_nombre_ultimo_escrapeo or url_viaje in lista_url_ultimo_escrapeo:
        row['en_ultimo_escrapeo'] = 'Si'
    else:
        row['en_ultimo_escrapeo'] = 'No'

In [77]:
#elimino duplicados
df_viajes_unicos = df_viajes_unicos.drop_duplicates().reset_index(drop=True)
len(df_viajes_unicos)

1435

In [79]:
#divido la columna duracion_viaje para extraer los días y las noches en diferentes columnas
df_viajes_unicos[['duracion_dias','duracion_noches']]=df_viajes_unicos['duracion_viaje'].str.split(" / ", expand = True)
df_viajes_unicos['duracion_dias']= (df_viajes_unicos['duracion_dias'].str.split(' ', expand = True)[0]).astype(int)
df_viajes_unicos['duracion_noches']= (df_viajes_unicos['duracion_noches'].str.split(' ', expand = True)[0]).astype(int)
df_viajes_unicos.columns

Index(['url_viaje', 'nombre_viaje', 'duracion_viaje',
       'itinerario_modificado_para_dividir', 'en_ultimo_escrapeo',
       'duracion_dias', 'duracion_noches'],
      dtype='object')

In [80]:
df_viajes_unicos.en_ultimo_escrapeo.value_counts()

en_ultimo_escrapeo
Si    1379
No      56
Name: count, dtype: int64

In [81]:
#convierto la columna en_ultimo_escrapeo en booleana
df_viajes_unicos = lo.convertir_si_no_a_boolean(df_viajes_unicos, 'en_ultimo_escrapeo')

In [82]:
#elimino la columna duracion_viaje
df_viajes_unicos= df_viajes_unicos.drop(columns='duracion_viaje')

In [83]:
#elimino duplicados
df_viajes_unicos = df_viajes_unicos.drop_duplicates().reset_index(drop=True)
len(df_viajes_unicos)

1435

In [84]:
print(len(df_viajes_unicos))
print(len(df_viajes_unicos.nombre_viaje.unique()))
print(len(df_viajes_unicos.url_viaje.unique()))

1435
1429
1435


### 5.2 Hago la carga en la BBDD:

La tabla viaje tiene que tener los siguientes campos:
- id_viaje SERIAL PRIMARY KEY,
- nombre_viaje TEXT UNIQUE,
- url_viaje TEXT UNIQUE,
- duracion_dias INT,
- duracion_noches INT,
- id_itinerario INT REFERENCES itinerario(id_itinerario) ON DELETE CASCADE,
- viaje_activo BOOLEAN


In [85]:
#Extraigo la información de la BBDD existente en la tabla viaje
query_extraccion = "SELECT url_viaje, id_viaje FROM viaje"
url_dict=lo.extraer_datos_de_BBDD(query_extraccion)
print(len(url_dict))

1435


In [86]:
#Extraigo la información de la BBDD existente en la tabla itinerarios
query_extraccion = "SELECT detalle_itinerario, id_itinerario FROM itinerario"
itinerario_dict=lo.extraer_datos_de_BBDD(query_extraccion)
print(len(itinerario_dict))

1139


In [87]:
query_extraccion_tupla="SELECT url_viaje, nombre_viaje, viaje_activo FROM viaje"
existing_tupla = lo.extraer_tupla_datos_bbdd(query_extraccion_tupla)  # set de tuplas (url_viaje, nombre_viaje,viaje_activo)

In [89]:
df_viajes_unicos

,url_viaje,nombre_viaje,itinerario_modificado_para_dividir,en_ultimo_escrapeo,duracion_dias,duracion_noches
0,https://es.tui.com/viaje/albania-y-montenegro,Albania y Montenegro,"Tirana, Dürres, Berat, Kruja, Shkodër, Petrova...",True,8,7
1,https://viajeonline.es.tui.com/es/idea/1463025...,Berlín,Berlín,True,4,3
2,https://viajeonline.es.tui.com/es/idea/7897512...,"Fly & Drive Austria, Entre Lagos y Montañas","Salzburgo, Gmunden, Linz, Melk, Viena, Graz, H...",True,8,7
3,https://viajeonline.es.tui.com/es/idea/8404511...,Fly & Drive Carlomagno a su aire,"Bruselas, Durbuy, Lieja, Aquisgrán, Colonia, B...",True,10,9
4,https://viajeonline.es.tui.com/es/idea/7906930...,Fly & Drive Tirol al Completo,"sbruck, Cataratas Krimml, Vipiteno, Bresanona,...",True,8,7
...,...,...,...,...,...,...
1430,https://viajeonline.es.tui.com/es/idea/2607762...,Laponia - Puente de Diciembre en Levi - desde ...,Levi,True,5,4
1431,https://viajeonline.es.tui.com/es/idea/2607572...,Laponia - Puente de Diciembre en Levi - desde ...,Levi,True,5,4
1432,https://viajeonline.es.tui.com/es/idea/2607664...,Laponia - Puente de Diciembre en Levi - desde ...,Levi,True,5,4
1433,https://viajeonline.es.tui.com/es/idea/2607847...,Laponia - Puente de Diciembre en Levi - desde ...,Levi,True,5,4


In [90]:
#Creo los datos para insertarlos en la BBDD:
data_to_insert=[]
data_to_update=[]
for _,row in df_viajes_unicos.iterrows():
    url_viaje = row["url_viaje"]
    if url_viaje not in url_dict.keys():
        nombre_viaje = row['nombre_viaje']
        duracion_dias = row['duracion_dias']
        duracion_noches = row['duracion_noches']
        detalle_itinerario = row['itinerario_modificado_para_dividir']
        id_itinerario = itinerario_dict.get(detalle_itinerario)
        viaje_activo = row['en_ultimo_escrapeo']
        data_to_insert.append([url_viaje,nombre_viaje,duracion_dias,duracion_noches,id_itinerario,viaje_activo])
    else:
        nombre_viaje = row['nombre_viaje']
        viaje_activo = row['en_ultimo_escrapeo']
        tupla_combinacion = (url_viaje, nombre_viaje, viaje_activo) #creo la tupla de combinación url_viaje, nombre_viaje, viaje_activo
        if tupla_combinacion not in existing_tupla:
            data_to_update.append([url_viaje,nombre_viaje,viaje_activo])

In [91]:

insert_query = """
INSERT INTO viaje (url_viaje,nombre_viaje,duracion_dias,duracion_noches,id_itinerario,viaje_activo)
VALUES (%s, %s, %s, %s, %s, %s)
"""
lo.insertar_datos_en_BBDD(insert_query, data_to_insert)

if (len(data_to_update))>0:
    for url_viaje, nombre_viaje, viaje_activo in data_to_update:
        update_query = """
        UPDATE viaje 
        SET nombre_viaje = %s,
            viaje_activo=%s
        where url_viaje = %s
        """
        lo.actualizar_datos_en_bbdd(update_query, ((nombre_viaje, viaje_activo, url_viaje)))
print(f'Se han cargado {len(data_to_insert)} viajes nuevos en la BBDD')
print(f'Se han actualizado {len(data_to_update)} viajes existentes en la BBDD')


Se han cargado 0 viajes nuevos en la BBDD
Se han actualizado 0 viajes existentes en la BBDD


## 6. Tabla viaje

### 6.1 Extraigo la información de los viajes procesados:

In [92]:
df_viajes_total.head()

,pais,nombre_viaje,duracion_viaje,itinerario,precio,url_viaje,fecha_escrapeo,en_ultimo_escrapeo,itinerario_modificado_para_dividir
0,Albania,Albania y Montenegro,8 Días / 7 Noches,"Tirana, Dürres, Berat, Kruja, Shkodër, Petrova...",1755,https://es.tui.com/viaje/albania-y-montenegro,2025-04-07,Si,"Tirana, Dürres, Berat, Kruja, Shkodër, Petrova..."
1,Alemania,Berlín,4 Días / 3 Noches,Berlín,708,https://viajeonline.es.tui.com/es/idea/1463025...,2025-04-07,Si,Berlín
2,Alemania,"Fly & Drive Austria, Entre Lagos y Montañas",8 Días / 7 Noches,"Salzburgo, Gmunden, Linz, Melk, Viena, Graz, H...",827,https://viajeonline.es.tui.com/es/idea/7897512...,2025-04-07,Si,"Salzburgo, Gmunden, Linz, Melk, Viena, Graz, H..."
3,Alemania,Fly & Drive Carlomagno a su aire,10 Días / 9 Noches,"Bruselas, Durbuy, Lieja, Aquisgrán, Colonia, B...",857,https://viajeonline.es.tui.com/es/idea/8404511...,2025-04-07,Si,"Bruselas, Durbuy, Lieja, Aquisgrán, Colonia, B..."
4,Alemania,Fly & Drive Tirol al Completo,8 Días / 7 Noches,"sbruck, Cataratas Krimml, Vipiteno, Bresanona,...",894,https://viajeonline.es.tui.com/es/idea/7906930...,2025-04-07,Si,"sbruck, Cataratas Krimml, Vipiteno, Bresanona,..."


In [93]:
#me quedo con las columnas que me interesan:
df_precio_viaje = df_viajes_total[['url_viaje','precio','fecha_escrapeo' ]]
df_precio_viaje

,url_viaje,precio,fecha_escrapeo
0,https://es.tui.com/viaje/albania-y-montenegro,1755,2025-04-07
1,https://viajeonline.es.tui.com/es/idea/1463025...,708,2025-04-07
2,https://viajeonline.es.tui.com/es/idea/7897512...,827,2025-04-07
3,https://viajeonline.es.tui.com/es/idea/8404511...,857,2025-04-07
4,https://viajeonline.es.tui.com/es/idea/7906930...,894,2025-04-07
...,...,...,...
55768,https://es.tui.com/viaje/cataratas-y-parques-d...,5369,2025-05-18
55769,https://es.tui.com/viaje/cataratas-y-parques-d...,5868,2025-05-18
55770,https://es.tui.com/viaje/cataratas-y-parques-d...,6189,2025-05-18
55771,https://es.tui.com/viaje/zimbabue-salvaje,6539,2025-05-18


### 6.2 Hago la carga en la BBDD:

La tabla precio_viaje tiene que tener los siguientes campos:
- id_registro_precio_viaje SERIAL PRIMARY KEY,
- precio_viaje INT,
- fecha_precio_viaje DATE,
- id_viaje INT REFERENCES viaje(id_viaje) ON DELETE CASCADE

In [101]:
#Extraigo la información de la BBDD existente en la tabla viaje
query_extraccion = "SELECT url_viaje, id_viaje FROM viaje"
url_dict=lo.extraer_datos_de_BBDD(query_extraccion)
print(len(url_dict))

1435


In [102]:
query_extraccion_tupla= "SELECT precio_viaje, fecha_precio_viaje, id_viaje FROM precio_viaje"
existing_tupla = lo.extraer_tupla_datos_bbdd(query_extraccion_tupla)  # set de tuplas (precio_viaje, fecha_precio_viaje, id_viaje)

In [103]:
#Creo los datos para insertarlos en la BBDD:
data_to_insert=[]
for _,row in df_precio_viaje.iterrows():
    url_viaje = row["url_viaje"]
    id_viaje = url_dict.get(url_viaje)
    precio_viaje = row['precio']
    fecha_precio_viaje = row['fecha_escrapeo'].date()
    tupla_combinacion = (precio_viaje, fecha_precio_viaje, id_viaje) #creo la tupla de combinación precio_viaje, fecha_precio_viaje, id_viaje
    if tupla_combinacion not in existing_tupla:
        data_to_insert.append([precio_viaje,fecha_precio_viaje,id_viaje])
print(len(data_to_insert))

0


In [104]:
insert_query = """
INSERT INTO precio_viaje (precio_viaje,fecha_precio_viaje,id_viaje)
VALUES (%s, %s, %s)
"""
lo.insertar_datos_en_BBDD(insert_query, data_to_insert)

## 7. Tabla combinacion_destino_viaje

### 7.1 Extraigo la información del df_viajes_total y me quedo con las columnas url_viaje y pais sin duplicados:

In [105]:
df_viajes_total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55773 entries, 0 to 55772
Data columns (total 9 columns):
 #   Column                              Non-Null Count  Dtype         
---  ------                              --------------  -----         
 0   pais                                55773 non-null  object        
 1   nombre_viaje                        55773 non-null  object        
 2   duracion_viaje                      55773 non-null  object        
 3   itinerario                          55773 non-null  object        
 4   precio                              55773 non-null  int64         
 5   url_viaje                           55773 non-null  object        
 6   fecha_escrapeo                      55773 non-null  datetime64[ns]
 7   en_ultimo_escrapeo                  55773 non-null  object        
 8   itinerario_modificado_para_dividir  55773 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(7)
memory usage: 3.8+ MB


In [106]:
#normalizo y capitalizo la columna pais:
df_viajes_total['pais'] = df_viajes_total['pais'].map(tr.normalizar_texto)
df_viajes_total['pais'] = df_viajes_total['pais'].map(tr.capitalizar_texto)

In [107]:
df_combinacion_destino_viaje = df_viajes_total[['url_viaje', 'pais']]

In [108]:
#eliminos duplicados
df_combinacion_destino_viaje = df_combinacion_destino_viaje.drop_duplicates()
df_combinacion_destino_viaje

,url_viaje,pais
0,https://es.tui.com/viaje/albania-y-montenegro,Albania
1,https://viajeonline.es.tui.com/es/idea/1463025...,Alemania
2,https://viajeonline.es.tui.com/es/idea/7897512...,Alemania
3,https://viajeonline.es.tui.com/es/idea/8404511...,Alemania
4,https://viajeonline.es.tui.com/es/idea/7906930...,Alemania
...,...,...
47175,https://viajeonline.es.tui.com/es/idea/2607762...,Laponia
47176,https://viajeonline.es.tui.com/es/idea/2607572...,Laponia
47177,https://viajeonline.es.tui.com/es/idea/2607664...,Laponia
47178,https://viajeonline.es.tui.com/es/idea/2607847...,Laponia


### 7.2 Hago la carga en la BBDD:

La tabla combinacion_destino_viaje tiene que tener los siguientes campos:
- id_viaje INT REFERENCES viaje(id_viaje) ON DELETE CASCADE,
- id_pais_destino INT REFERENCES pais_destino(id_pais_destino) ON DELETE CASCADE,
- PRIMARY KEY (id_viaje, id_pais_destino)

In [109]:
#Extraigo la información de la BBDD existente en la tabla viaje
query_extraccion = "SELECT url_viaje, id_viaje FROM viaje"
viaje_dict=lo.extraer_datos_de_BBDD(query_extraccion)
print(len(ciudad_dict))

2040


In [110]:
#Extraigo la información de la BBDD existente en la tabla pais_destino:
query_extraccion = "SELECT nombre_pais_destino, id_pais_destino FROM pais_destino"
pais_dict=lo.extraer_datos_de_BBDD(query_extraccion)
print(len(pais_dict))

187


In [111]:
# Obtengo todas las combinaciones existentes (id_viaje, id_pais_destino) de la tabla
query_extraccion_tupla= "SELECT id_viaje, id_pais_destino FROM combinacion_destino_viaje"
existing_pairs = lo.extraer_tupla_datos_bbdd(query_extraccion_tupla)  # set de tuplas (id_viaje, id_pais_destino)

In [112]:
#Creo los datos para insertarlos en la BBDD:
data_to_insert=[]
for _,row in df_combinacion_destino_viaje.iterrows():
    url_viaje = row["url_viaje"]
    pais_destino = row["pais"]
    id_viaje = viaje_dict.get(url_viaje)
    id_pais_destino = pais_dict.get(pais_destino)
    pair = (id_viaje, id_pais_destino) #creo la tupla de combinación de id_viaje, id_pais_destino
    if pair not in existing_pairs:
        data_to_insert.append([id_viaje, id_pais_destino])
print(len(data_to_insert))

0


In [113]:
#Subo la información a la BBDD:
insert_query = """
INSERT INTO combinacion_destino_viaje (id_viaje,id_pais_destino)
VALUES (%s, %s)
"""
lo.insertar_datos_en_BBDD(insert_query,data_to_insert)
print(f'Se han añadido {len(data_to_insert)} registros en la tabla itinerariocombinacion_destino_viaje')

Se han añadido 0 registros en la tabla itinerariocombinacion_destino_viaje


## 8. Tabla turismo_emisor

### 8.1 Extraigo la información del fichero extraido de la API de turismo emisor:

In [114]:
df_turismo_emisor_procesado

,Unnamed: 0,AÑO,MES,CCAA_ORIGEN,CONTINENTE_DESTINO,PAIS_DESTINO,TURISTAS,PERNOCTACIONES,ESTANCIA_MEDIA
0,0,2019,7,Andalucía,América,Argentina,"419,0","3616,0","8,6"
1,1,2019,7,Andalucía,América,Bolivia,"61,0","486,0","8,0"
2,2,2019,7,Andalucía,América,Brasil,"277,0","2214,0","8,0"
3,3,2019,7,Andalucía,América,Canadá,"483,0","6159,0","12,8"
4,4,2019,7,Andalucía,América,Chile,"242,0","2068,0","8,5"
...,...,...,...,...,...,...,...,...,...
71092,81658,2024,12,Total Nacional,África,Seychelles,"146,0","1105,0","7,6"
71093,81659,2024,12,Total Nacional,África,Sudáfrica,"986,0","9019,0","9,1"
71094,81660,2024,12,Total Nacional,África,Tanzania,"590,0","5237,0","8,9"
71095,81661,2024,12,Total Nacional,África,Túnez,"975,0","5012,0","5,1"


In [115]:
#normalizo y capitalizo la columna PAIS_DESTINO:
df_turismo_emisor_procesado['PAIS_DESTINO'] = df_turismo_emisor_procesado['PAIS_DESTINO'].map(tr.normalizar_texto)
df_turismo_emisor_procesado['PAIS_DESTINO'] = df_turismo_emisor_procesado['PAIS_DESTINO'].map(tr.capitalizar_texto)

In [132]:
#transformo las columnas pernoctaciones y turistas en enteros:
for column in ['PERNOCTACIONES', 'TURISTAS']:
    df_turismo_emisor_procesado[column]=df_turismo_emisor_procesado[column].str.replace(',','.').astype(float).astype(int)

In [133]:
#construyo el DF con los datos que me interesa:
df_turismo_emisor_unico = df_turismo_emisor_procesado[['CCAA_ORIGEN','PAIS_DESTINO', 'TURISTAS','PERNOCTACIONES','AÑO','MES' ]]

In [134]:
df_turismo_emisor_unico.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71097 entries, 0 to 71096
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   CCAA_ORIGEN     71097 non-null  object
 1   PAIS_DESTINO    71097 non-null  object
 2   TURISTAS        71097 non-null  int64 
 3   PERNOCTACIONES  71097 non-null  int64 
 4   AÑO             71097 non-null  int64 
 5   MES             71097 non-null  int64 
dtypes: int64(4), object(2)
memory usage: 3.3+ MB


### 8.2 Hago la carga en la BBDD:

La tabla turismo_emisor tiene que tener los siguientes campos:
- id_registro_turismo SERIAL PRIMARY KEY,
- ccaa_origen TEXT,
- id_pais_destino INT REFERENCES pais_destino(id_pais_destino) ON DELETE CASCADE,
- num_turistas INT,
- num_pernoctaciones INT,
- ano INT,
- mes INT

In [139]:
#extraigo los datos cargados en la BBDD para no duplicar los datos en las futuras subidas:
query_extraccion_tupla= "SELECT ccaa_origen, id_pais_destino, num_turistas, num_pernoctaciones, ano, mes FROM turismo_emisor"
existing_tupla = lo.extraer_tupla_datos_bbdd(query_extraccion_tupla)  # set de tuplas (ccaa_origen, id_pais_destino, num_turistas, num_pernoctaciones, ano, mes)
print(len(existing_tupla))

71097


In [140]:
#Extraigo el id_pais_destino de la tabla pais_destino de la BBDD:
query_extraccion = "SELECT nombre_pais_destino, id_pais_destino FROM pais_destino"
pais_dict=lo.extraer_datos_de_BBDD(query_extraccion)
print(len(pais_dict))

187


In [141]:
#Creo los datos para insertarlos en la BBDD:
data_to_insert=[]
for _,row in df_turismo_emisor_unico.iterrows():
    ccaa_origen = row["CCAA_ORIGEN"]
    pais_destino = row['PAIS_DESTINO']
    id_pais_destino = pais_dict.get(pais_destino)
    num_turistas = row['TURISTAS']
    num_pernoctaciones = row['PERNOCTACIONES']
    ano = row['AÑO']
    mes = row['MES']
    pair = (ccaa_origen, id_pais_destino, num_turistas, num_pernoctaciones, ano, mes) #creo la tupla de combinación
    if pair not in existing_tupla:
        data_to_insert.append([ccaa_origen, id_pais_destino, num_turistas, num_pernoctaciones, ano, mes])
print(len(data_to_insert))

0


In [142]:
#Subo la información a la BBDD:
insert_query = """
INSERT INTO turismo_emisor (ccaa_origen, id_pais_destino, num_turistas, num_pernoctaciones, ano, mes)
VALUES (%s, %s, %s, %s, %s, %s)
"""
lo.insertar_datos_en_BBDD(insert_query,data_to_insert)
print(f'Se han añadido {len(data_to_insert)} registros en la tabla turismo_emisor')

Se han añadido 0 registros en la tabla turismo_emisor
